In [1]:
%load_ext autoreload
%autoreload 2

In [80]:
import numpy as np
import polars
from polars import DataFrame, col
from transformers import AutoModelForSequenceClassification
from label_legends.deberta import load_dataset, load_deberta, load_dataset
from label_legends.preprocess import holdout, load_data, load_test, transform
from label_legends.female import predict_female
from label_legends.result import calculate_scores

# Use Deberta to create initial predictions:

In [3]:
deberta = load_deberta()

/home/lukas/.local/share/uv/python/cpython-3.12.5-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

  self.pid = os.fork()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/home/lukas/.local/share/uv/python/cpython-3.12.5-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

  self.pid = os.fork()


/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
train_out = deberta.train()
train_out.metrics

/home/lukas/Programming/uni/nlp-ie-label-legends/src/label_legends/deberta.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss


/home/lukas/Programming/uni/nlp-ie-label-legends/src/label_legends/deberta.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


🏃 View run ./results at: https://mlflow.mahluke.page/#/experiments/0/runs/3d83c6467ac34c4698698edfd01c17fa
🧪 View experiment at: https://mlflow.mahluke.page/#/experiments/0


{'train_runtime': 730.5862,
 'train_samples_per_second': 40.242,
 'train_steps_per_second': 2.012,
 'total_flos': 1405099900663200.0,
 'train_loss': 0.4091674350556873,
 'epoch': 1.0}

In [7]:
test = load_test().collect()
test_transformed = transform(test)
deberta_prediction = deberta.predict(load_dataset(test_transformed["text"].to_list(), test_transformed["label"].to_list()))
deberta_prediction

/home/lukas/Programming/uni/nlp-ie-label-legends/src/label_legends/deberta.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


PredictionOutput(predictions=array([[ 0.16233578, -0.09792556],
       [ 1.48139   , -1.5975136 ],
       [ 1.48139   , -1.5975136 ],
       ...,
       [ 0.22276703, -0.18155377],
       [ 0.22276703, -0.18155377],
       [ 0.22276703, -0.18155377]], dtype=float32), label_ids=array([1, 0, 0, ..., 1, 0, 1]), metrics={'test_loss': 0.3546888828277588, 'test_precision': 0.7497103128621089, 'test_recall': 0.6231139646869984, 'test_fscore': 0.6805750350631136, 'test_accuracy': 0.8481666666666666, 'test_tp': 1941, 'test_tn': 8237, 'test_fp': 648, 'test_fn': 1174, 'test_runtime': 77.6839, 'test_samples_per_second': 154.472, 'test_steps_per_second': 7.724})

In [24]:
predictions = DataFrame({"id": test["id"], "label": deberta_prediction.label_ids, "deberta": np.argmax(deberta_prediction.predictions, axis=1)})
predictions.head()

id,label,deberta
i64,i64,i64
100,1,0
10005,0,0
10006,0,0
10007,0,0
10008,0,0


In [81]:
scores_deberta = calculate_scores(predictions["label"], predictions["deberta"])

precision:	0.7497
recall:		0.6231
fscore:		0.6806
accuracy:	0.8482
tn: 8237	 fp: 648
fn: 1174	 tp: 1941

# Post-Process negative predictions

## Predict if sentence is referencing a female

In [85]:
def assign_type(prediction: int, label: str):
    if prediction == 0:
        if label == "not sexist":
            return "tn"
        return "fn"
    if label == "sexist":
        return "tp"
    return "fp"


predictions = predictions.join(test, on="id").with_columns(polars.struct(["deberta", "label_sexist"]).map_elements(lambda x: assign_type(x['deberta'],x['label_sexist']), return_dtype=polars.String).alias("type")).select(["id", "type", "label", "deberta"])

predictions.head()

id,type,label,deberta
i64,str,i64,i64
100,"""fn""",1,0
10005,"""tn""",0,0
10006,"""tn""",0,0
10007,"""tn""",0,0
10008,"""tn""",0,0


Add the predictions whether a text is about a female or not:

In [86]:
def alt_on_neg_pred(prediction: int, alternative: int):
    """Use the alternative if prediction is negative"""
    if prediction == 1:
        return 1
    return alternative


predictions = predictions.join(predict_female(test), on="id").with_columns(
    polars.struct(["female", "deberta"]).map_elements(lambda x: alt_on_neg_pred(x["deberta"], x["female"])).alias("pred_female"))

/tmp/ipykernel_75696/2309025452.py:8: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  predictions = predictions.join(predict_female(test), on="id").with_columns(


We see that the prediction of most false positives is reconsidered, as those contain some female phrase. However, many of true negatives are also reconsidered, leading to a potentially worse precision. As we are focusing on increasing the recall, this is a trade-off we have to make.

In [77]:
predictions.group_by("female", "type").len().with_columns((polars.col("len") / polars.sum("len").over("type")).alias("perc")).filter(col("female")==1).sort("len").rename({"len": "samples"})

female,type,samples,perc
i32,str,u32,f64
1,"""fp""",575,0.887346
1,"""fn""",1043,0.888416
1,"""tp""",1657,0.853684
1,"""tn""",5563,0.675367


In [87]:
predictions.head()

id,type,label,deberta,female,pred_female
i64,str,i64,i64,i32,i64
100,"""fn""",1,0,1,1
10005,"""tn""",0,0,1,1
10006,"""tn""",0,0,1,1
10007,"""tn""",0,0,1,1
10008,"""tn""",0,0,1,1


If we would only use 'female' without some additional measurements, we would increase our recall considerably to over 95%, which comes at the cost of precision and f-score, which are drastically decreased. In order to not worsen the model that strongly overall, we add additional measurements to decide when to change a negative prediction by Deberta to a positive one.

In [88]:
scores_female = calculate_scores(predictions["label"], predictions["pred_female"])
scores_female

precision:	0.3245
recall:		0.9579
fscore:		0.4848
accuracy:	0.4715
tn: 2674	 fp: 6211
fn: 131	 tp: 2984

## Sentiment analysis
Predict if text has a negative connotation. If true, and also targeted towards a female, predict the sample to be sexist.




## Bad words
Check if text contains a 'bad word', if so, predict the sample to be sexist.